# Import Necessary Libraries

In [4]:
import collections
import numpy as np
import nltk
from nltk.corpus import treebank
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [5]:
# get the corpus
import random
sentences=list(treebank.tagged_sents(tagset='universal'))
random.shuffle(sentences)

In [6]:
#Splitting the data for train and test
split_num_train = int(len(sentences)*0.8)
split_num_valid = int(len(sentences)*0.9)
train_data = sentences[0:split_num_train]
valid_data = sentences[split_num_train:split_num_valid]
test_data  = sentences[split_num_valid:]

# Build a Vocabulary

In [7]:
def build_vocab(corpus, freq):
  # get the unique words and tags
  words=[]
  tags=[]
  for sent in corpus:
    for tokens in sent:
      words.append(tokens[0])
      tags.append(tokens[1])
  tag_cols=list(set(tags))
  tag_cols.sort()
  
  # count the word freqency
  word_counts=collections.Counter(words).most_common() # sort the word in dictionary by their frequencies.
  idx=0
  vocab={}
  for word, counts in word_counts:
    if counts>freq: # Set a boundray to add only words with frequency greater than specefied frequency.
      vocab[word]=idx
      idx+=1
  vocab['UNK']=idx #add an Unknown tag at the end of the vocab.
  return vocab, words, tag_cols

In [58]:
# build vocab
vocab, words, tag_cols=build_vocab(corpus=train_data, freq=3)
tag_cols.insert(0, 'START') 
tag_cols.insert(len(tag_cols), 'END')

# Calculate Transition Matrix

In [9]:
def compute_transition_matrix(corpus, tag_cols):
  # get the tags
  tags_in_line=[]
  for sent in corpus:
    tags_per_line=[]
    for tokens in sent:
      tags_per_line.append(tokens[1])
    tags_in_line.append(tags_per_line)
  #print(tags_in_line)
  # compute the transition counts matrix
  cor_matrix=np.zeros((len(tag_cols), len(tag_cols)))
  for tags in tags_in_line:
    for i in range(len(tags)):
      if i==0:
        idx_x=tag_cols.index('START')
        idx_y=tag_cols.index(tags[i])
      elif i==len(tags)-1:
        idx_x=tag_cols.index(tags[i])
        idx_y=tag_cols.index('END')
      else:
        idx_x=tag_cols.index(tags[i])     # an arbitrary index
        idx_y=tag_cols.index(tags[i+1])   # next index
      cor_matrix[idx_x][idx_y]+=1
  return cor_matrix


  # compute the transition **counts matrix**


In [10]:
import pandas as pd

trans_matrix=compute_transition_matrix(corpus=train_data, tag_cols=tag_cols)
pd.DataFrame(trans_matrix , columns = tag_cols , index = tag_cols )

,START,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X,END
START,0.0,262.0,132.0,410.0,159.0,167.0,726.0,917.0,28.0,223.0,3.0,29.0,75.0,0.0
.,0.0,616.0,268.0,447.0,320.0,375.0,817.0,1178.0,715.0,314.0,18.0,801.0,176.0,3107.0
ADJ,0.0,321.0,328.0,376.0,25.0,77.0,23.0,3496.0,102.0,3.0,54.0,56.0,102.0,0.0
ADP,0.0,306.0,785.0,127.0,101.0,6.0,2410.0,2447.0,476.0,521.0,10.0,63.0,264.0,2.0
ADV,0.0,281.0,341.0,287.0,199.0,17.0,173.0,69.0,84.0,30.0,36.0,854.0,59.0,1.0
CONJ,0.0,53.0,203.0,73.0,104.0,1.0,168.0,607.0,76.0,81.0,6.0,278.0,8.0,0.0
DET,0.0,124.0,1313.0,57.0,86.0,3.0,33.0,3959.0,133.0,23.0,1.0,208.0,333.0,1.0
NOUN,0.0,5503.0,271.0,4046.0,387.0,992.0,309.0,5533.0,211.0,104.0,989.0,3216.0,670.0,16.0
NUM,0.0,339.0,94.0,101.0,8.0,37.0,10.0,983.0,515.0,4.0,70.0,42.0,583.0,2.0
PRON,0.0,83.0,153.0,48.0,53.0,9.0,19.0,444.0,14.0,16.0,32.0,919.0,182.0,0.0


  # compute the emission **counts matrix**


In [11]:
def compute_emission_matrix(corpus, vocab, tag_cols):
  # compute the emission counts matrix
  cor_matrix=np.zeros((len(tag_cols), len(vocab.keys())))
  for sent in corpus:
    for tokens in sent:
      idx_x=tag_cols.index(tokens[1])
      if tokens[0] in vocab.keys():
        idx_y=vocab[tokens[0]]
      else:
        idx_y=vocab['UNK']
      cor_matrix[idx_x][idx_y]+=1
  return cor_matrix

In [12]:
# get emission table
emission_matrix=compute_emission_matrix(corpus=train_data, vocab=vocab, tag_cols=tag_cols)

In [13]:
pd.DataFrame(emission_matrix , index = tag_cols)

,0,1,2,3,4,5,6,7,8,9,...,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592
START,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.,3954.0,0.0,3066.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
ADJ,0.0,5.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1515.0
ADP,0.0,0.0,0.0,1857.0,1.0,1.0,1265.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
ADV,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,365.0
CONJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1237.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
DET,0.0,3242.0,0.0,0.0,0.0,1510.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
NOUN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,6171.0
NUM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,888.0
PRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0


# Now Convert to Probability

## Transition probability

In [14]:
def estimate_transition_prob(y_pos, y_pre, trans_matrix, tag_cols, beta=0):
  idx_x=tag_cols.index(y_pre)
  idx_y=tag_cols.index(y_pos)
  p=(trans_matrix[idx_x][idx_y]+beta)/(np.sum(trans_matrix[idx_x])+len(tag_cols)*beta)
  return p

## Emission Probability

In [15]:
def estimate_emission_prob(c_word, c_tag , emission_matrix, vocab, tag_cols, alpha=0):
  idx_x=tag_cols.index(c_tag)
  if c_word in vocab.keys():
    idx_y=vocab[c_word]
  else:
    idx_y=vocab['UNK']
  p=(emission_matrix[idx_x][idx_y]+alpha)/(np.sum(emission_matrix[idx_x])+alpha*len(vocab.keys()))
  return p

## Define Viterbi Algorithm

In [16]:
def viterbi(sentence, vocab, tag_cols, trans_matrix, emission_matrix, alpha, beta):
  v=np.zeros((len(sentence)+1, len(tag_cols)-2)) # Hold best probabilities
  b=np.zeros((len(sentence)+1, len(tag_cols)-2)) # Hold the indexes of best probabilities.
  s=np.zeros((1, len(tag_cols)-2))
  ### calculate s(y0, START), v(x0) ###
  for k in range(1, len(tag_cols)-1): # Ignoring "Start" , "End"
    tp=estimate_transition_prob(tag_cols[k], 'START', trans_matrix=trans_matrix, 
                                tag_cols=tag_cols, beta=beta)
    ep=estimate_emission_prob(sentence[0], tag_cols[k], emission_matrix, vocab, tag_cols, alpha)
    v[0 , k-1]=np.log(tp)+np.log(ep) # log (tp * ep)
    b[0 , k-1]=tag_cols.index('START')

  ### calculate s(END, yi), v(END) ###
  for k in range(1, len(tag_cols)-1):
    for kk in range(1, len(tag_cols)-1):
      tp=estimate_transition_prob('END', tag_cols[kk], trans_matrix, tag_cols, beta)
      ep=1
      s[0 , kk-1]=np.log(tp)+np.log(ep)

    v[len(sentence) , k-1]=np.max(v[len(sentence)-1]+s[0])      # Max Probability 
    b[len(sentence) , k-1]=np.argmax(v[len(sentence)-1]+s[0])+1 # Max Index
  

  ### calculate s(yi, yi_1), v(xi) ###
  for m in range(1, len(sentence)):
    for k in range(1, len(tag_cols)-1):
      for kk in range(1, len(tag_cols)-1):

        tp=estimate_transition_prob(tag_cols[k], tag_cols[kk], trans_matrix, tag_cols, beta)
        ep=estimate_emission_prob(sentence[m], tag_cols[k], emission_matrix, vocab, tag_cols, alpha)

        s[0 , kk-1]=np.log(tp)+np.log(ep)

      v[m , k-1]=np.max(v[m-1]+s[0])
      b[m , k-1]=np.argmax(v[m-1]+s[0])+1    # plus 1 to align with the index in tag_cols
    

    
  # get the predict tags
  m_idx=np.array(np.arange(1, len(sentence)))
  m_idx=m_idx[::-1]
  y_m=[]
  y_m.append(tag_cols[int(b[len(sentence)][0])])
  for i, m in enumerate(m_idx):
    b_last=tag_cols.index(y_m[i])
    b_now=b[m][int(b_last)-1]
    y_m.append(tag_cols[int(b_now)])
  y_m.reverse() 
  return y_m, v, b

In [55]:
def get_dev_acc(corpus, alpha, beta, vocab, tag_cols, trans_matrix, emission_matrix):
  acc_num=0.
  total_num=0.
  for sent in corpus:
    words=[]
    label=[]
    for tokens in sent:
      words.append(tokens[0])
      label.append(tokens[1])
    preds, v, b=viterbi(sentence=words, vocab=vocab, tag_cols=tag_cols, 
                        trans_matrix=trans_matrix, emission_matrix=emission_matrix, 
                        alpha=alpha, beta=beta)      
    if len(corpus)==1:
      print('###########')
      print(f" The sentence : {' '.join(words)}\n")
      print(f" Actual labels : {list(zip(words, label))}\n")
      print(f" Predicted lables : {list(zip(words, preds))}\n")  

    for i , (pred, truth) in enumerate(zip(preds, label)):
      if pred==truth:
        acc_num+=1
      else :
        if len(corpus) == 1:
          print(f" Actual tag : {(words[i] , truth)}, predicted tag :{(words[i] , pred)}\n")
        

    total_num+=len(label)
  return acc_num/total_num

# Evaluate on dev set

In [17]:
Acc = np.zeros((3 , 3))
for i,a in enumerate(np.linspace(0.01 , 0.03 , 3)):
  for j,b in enumerate(np.linspace(1 , 3 , 3)):
    Acc[i , j] =get_dev_acc(corpus = valid_data, alpha=a, beta=b, vocab=vocab, tag_cols=tag_cols, 
                    trans_matrix=trans_matrix, emission_matrix=emission_matrix)
    print(f'alpha={a}, beta={b}, overal_accuracy={Acc[i , j]}')

alpha=0.01, beta=1.0, overal_accuracy=0.9197757390417941
alpha=0.01, beta=2.0, overal_accuracy=0.9197757390417941
alpha=0.01, beta=3.0, overal_accuracy=0.9196738022426095
alpha=0.019999999999999997, beta=1.0, overal_accuracy=0.9200815494393476
alpha=0.019999999999999997, beta=2.0, overal_accuracy=0.9200815494393476
alpha=0.019999999999999997, beta=3.0, overal_accuracy=0.9198776758409786
alpha=0.03, beta=1.0, overal_accuracy=0.9198776758409786
alpha=0.03, beta=2.0, overal_accuracy=0.9195718654434251
alpha=0.03, beta=3.0, overal_accuracy=0.919367991845056


In [18]:
Acc.argmax() #=> alpha = 0.01 , beta = 2

3

# Generate some outputs 

In [56]:
for i in np.random.randint(0 , len(valid_data) , 5):
  acc = get_dev_acc(corpus = [test_data[i]], alpha=0.01, beta=2, vocab=vocab, tag_cols=tag_cols, 
                trans_matrix=trans_matrix, emission_matrix=emission_matrix)
  print(f'\nalpha={0.01}, beta={2}, overal_accuracy={acc}\n')

###########
 The sentence : Other paper and forest-products stocks closed *-1 mixed .

 Actual labels : [('Other', 'ADJ'), ('paper', 'NOUN'), ('and', 'CONJ'), ('forest-products', 'NOUN'), ('stocks', 'NOUN'), ('closed', 'VERB'), ('*-1', 'X'), ('mixed', 'VERB'), ('.', '.')]

 Predicted lables : [('Other', 'NOUN'), ('paper', 'NOUN'), ('and', 'CONJ'), ('forest-products', 'ADJ'), ('stocks', 'NOUN'), ('closed', 'VERB'), ('*-1', 'X'), ('mixed', 'VERB'), ('.', '.')]

 Actual tag : ('Other', 'ADJ'), predicted tag :('Other', 'NOUN')

 Actual tag : ('forest-products', 'NOUN'), predicted tag :('forest-products', 'ADJ')


alpha=0.01, beta=2, overal_accuracy=0.7777777777777778

###########
 The sentence : PRECIOUS METALS : Futures prices eased as increased stability and strength came into the securities markets .

 Actual labels : [('PRECIOUS', 'NOUN'), ('METALS', 'NOUN'), (':', '.'), ('Futures', 'NOUN'), ('prices', 'NOUN'), ('eased', 'VERB'), ('as', 'ADV'), ('increased', 'VERB'), ('stability', 'NOU

# Find the accuracy on the test data

In [34]:
acc = get_dev_acc(corpus = test_data, alpha=0.01, beta=2, vocab=vocab, tag_cols=tag_cols, 
                trans_matrix=trans_matrix, emission_matrix=emission_matrix)
print(f'alpha={0.01}, beta={2}, overal_accuracy={acc}')

alpha=0.01, beta=2, overal_accuracy=0.9115158753856872
